# ⚡ ResNet-50 + Attention for Facial Emotion Recognition

This notebook:
- Loads pretrained ResNet-50
- Adds spatial attention (CBAM-style)
- Fine-tunes with FER-2013 dataset

In [3]:
#%pip uninstall torch torchvision torchaudio -y

%pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 -f https://download.pytorch.org/whl/torch_stable.html


Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ---------------------------------------- 0.0/2.1 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.1 GB 4.8 MB/s eta 0:07:26
     ---------------------------------------- 0.0/2.1 GB 4.8 MB/s eta 0:07:27
     ---------------------------------------- 0.0/2.1 GB 5.0 MB/s eta 0:07:10
     ---------------------------------------- 0.0/2.1 GB 5.2 MB/s eta 0:06:49
     ---------------------------------------- 0.0/2.1 GB 5.2 MB/s eta 0:06:48
     ---------------------------------------- 0.0/2.1 GB 5.4 MB/s eta 0:06:33
     ---------------------------------------- 0.0/2.1 GB 5.6 MB/s eta 0:06:22
     ---------------------------------------- 0.0/2.1 GB 5.6 MB/s eta 0:06:19
     ---------------------------------------- 0.0/2.1 GB 5.7 MB/s eta 0:06:15
     ---------------------------------------- 0.0/2.1 GB 5.8 MB/s eta 0:06:10
     ---------------------------------------- 0.0/2.1 GB 5.9 MB/s eta 0:06:04
  

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [2]:

# Dataset and transforms
train_dir = 'Data/train'
test_dir = 'Data/test'

train_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(5),
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

full_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
val_size = int(0.2 * len(full_dataset))
train_size = len(full_dataset) - val_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
val_dataset.dataset.transform = test_transform
test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

class_names = full_dataset.classes


In [3]:

class SpatialAttention(nn.Module):
    def __init__(self):
        super(SpatialAttention, self).__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size=7, padding=3)

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x_cat = torch.cat([avg_out, max_out], dim=1)
        attention = torch.sigmoid(self.conv(x_cat))
        return x * attention


In [4]:

# Build ResNet-50 with spatial attention block
class ResNet50WithAttention(nn.Module):
    def __init__(self, num_classes=7):
        super(ResNet50WithAttention, self).__init__()
        self.base_model = models.resnet50(pretrained=True)

        # Remove last FC layer
        self.base_model.fc = nn.Identity()

        # Add attention
        self.attention = SpatialAttention()

        # Final classification layer
        self.classifier = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(2048, num_classes)
        )

    def forward(self, x):
        x = self.base_model.conv1(x)
        x = self.base_model.bn1(x)
        x = self.base_model.relu(x)
        x = self.base_model.maxpool(x)

        x = self.base_model.layer1(x)
        x = self.base_model.layer2(x)
        x = self.base_model.layer3(x)
        x = self.base_model.layer4(x)

        x = self.attention(x)
        x = self.base_model.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

model = ResNet50WithAttention().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)


c:\Users\paghe\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\Users\paghe\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\paghe/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100.0%


In [5]:

# Train model
num_epochs = 20
train_losses = []
val_accuracies = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    scheduler.step()
    epoch_loss = running_loss / len(train_loader)
    train_losses.append(epoch_loss)

    # Validation
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.numpy())

    val_acc = accuracy_score(all_labels, all_preds)
    val_accuracies.append(val_acc)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Val Accuracy: {val_acc:.4f}")


Epoch [1/20], Loss: 1.1743, Val Accuracy: 0.6222
Epoch [2/20], Loss: 0.9009, Val Accuracy: 0.6443
Epoch [3/20], Loss: 0.7146, Val Accuracy: 0.6360
Epoch [4/20], Loss: 0.5062, Val Accuracy: 0.6185
Epoch [5/20], Loss: 0.3129, Val Accuracy: 0.6574
Epoch [6/20], Loss: 0.1011, Val Accuracy: 0.6753
Epoch [7/20], Loss: 0.0414, Val Accuracy: 0.6715
Epoch [8/20], Loss: 0.0361, Val Accuracy: 0.6675
Epoch [9/20], Loss: 0.0538, Val Accuracy: 0.6523
Epoch [10/20], Loss: 0.0460, Val Accuracy: 0.6684
Epoch [11/20], Loss: 0.0215, Val Accuracy: 0.6758
Epoch [12/20], Loss: 0.0125, Val Accuracy: 0.6779
Epoch [13/20], Loss: 0.0104, Val Accuracy: 0.6748
Epoch [14/20], Loss: 0.0103, Val Accuracy: 0.6791
Epoch [15/20], Loss: 0.0148, Val Accuracy: 0.6790
Epoch [16/20], Loss: 0.0075, Val Accuracy: 0.6835
Epoch [17/20], Loss: 0.0052, Val Accuracy: 0.6830
Epoch [18/20], Loss: 0.0054, Val Accuracy: 0.6764
Epoch [19/20], Loss: 0.0054, Val Accuracy: 0.6788
Epoch [20/20], Loss: 0.0053, Val Accuracy: 0.6797


In [6]:

# Test evaluation
model.eval()
test_preds, test_labels = [], []
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_preds.extend(predicted.cpu().numpy())
        test_labels.extend(labels.numpy())

test_acc = accuracy_score(test_labels, test_preds)
print(f"Test Accuracy: {test_acc:.4f}")


Test Accuracy: 0.6737


In [7]:

# Predict single image
test_image_path = 'my_image2.jpg'

image = Image.open(test_image_path).convert('L').convert('RGB')
image = test_transform(image).unsqueeze(0).to(device)

model.eval()
with torch.no_grad():
    output = model(image)
    probs = torch.softmax(output, dim=1)
    _, predicted = torch.max(probs, 1)
    predicted_class = class_names[predicted.item()]
    confidence = probs[0][predicted.item()].item()

print(f"Predicted Emotion: {predicted_class} ({confidence*100:.2f}% confidence)")


Predicted Emotion: surprise (99.98% confidence)
